In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [59]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

X_train = train_df.iloc[:, :-1].values.astype(np.float32)
y_train = train_df["target"].values.astype(np.int64)

X_test = test_df.iloc[:, 1:].values.astype(np.float32)

In [60]:
class MaxDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

train_data = MaxDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

In [61]:
class MaxNet(nn.Module):
    def __init__(self):
        super(MaxNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(50, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        return self.model(x)


In [62]:
model = MaxNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 1.6560
Epoch 2, Loss: 0.7731
Epoch 3, Loss: 0.2633
Epoch 4, Loss: 0.1224
Epoch 5, Loss: 0.0731
Epoch 6, Loss: 0.0470
Epoch 7, Loss: 0.0325
Epoch 8, Loss: 0.0240
Epoch 9, Loss: 0.0186
Epoch 10, Loss: 0.0146
Epoch 11, Loss: 0.0117
Epoch 12, Loss: 0.0093
Epoch 13, Loss: 0.0076
Epoch 14, Loss: 0.0062
Epoch 15, Loss: 0.0053
Epoch 16, Loss: 0.0043
Epoch 17, Loss: 0.0036
Epoch 18, Loss: 0.0030
Epoch 19, Loss: 0.0025
Epoch 20, Loss: 0.0022
Epoch 21, Loss: 0.0018
Epoch 22, Loss: 0.0016
Epoch 23, Loss: 0.0014
Epoch 24, Loss: 0.0012
Epoch 25, Loss: 0.0011
Epoch 26, Loss: 0.0010
Epoch 27, Loss: 0.0009
Epoch 28, Loss: 0.0008
Epoch 29, Loss: 0.0007
Epoch 30, Loss: 0.0006
Epoch 31, Loss: 0.0006
Epoch 32, Loss: 0.0005
Epoch 33, Loss: 0.0005
Epoch 34, Loss: 0.0004
Epoch 35, Loss: 0.0004
Epoch 36, Loss: 0.0003
Epoch 37, Loss: 0.0003
Epoch 38, Loss: 0.0003
Epoch 39, Loss: 0.0003
Epoch 40, Loss: 0.0002
Epoch 41, Loss: 0.0002
Epoch 42, Loss: 0.0002
Epoch 43, Loss: 0.0002
Epoch 44, Loss: 0.00

In [64]:
torch.save(model.state_dict(), "model.pth")

In [65]:
model = MaxNet()
model.load_state_dict(torch.load("model.pth", map_location=torch.device("cpu")))
model.eval()

MaxNet(
  (model): Sequential(
    (0): Linear(in_features=50, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [67]:
test_df = pd.read_csv("test.csv")
X_test = test_df.iloc[:, 1:].values.astype(np.float32)

In [68]:
with torch.no_grad():
    X_tensor = torch.tensor(X_test)
    outputs = model(X_tensor)
    preds = outputs.argmax(dim=1).numpy()

In [69]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "target": preds
})

submission.to_csv("submission.csv", index=False)
print("submission.csv is done")

submission.csv is done
